<a href="https://colab.research.google.com/github/victorbahlangene/Pythorch-V-fastai2-v-miniai/blob/main/1_model_using_raw_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq git+https://github.com/fastai/course22p2

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.1/989.1 kB 14.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 33.0 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import ToTensor

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# 1. Data: Pytorch data

In [15]:
tr_dset = torchvision.datasets.FashionMNIST(root="./data",
                                            train = True,
                                            transform = ToTensor(),
                                            download = True)
tst_dset = torchvision.datasets.FashionMNIST(root="./data",
                                            train = False,
                                            transform = ToTensor())

100%|██████████| 26421880/26421880 [00:00<00:00, 116393136.41it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 5481773.13it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw




100%|██████████| 4422102/4422102 [00:00<00:00, 65783437.16it/s]


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 21086208.00it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [16]:
tr_dset.data.shape, tst_dset.data.shape

(torch.Size([60000, 28, 28]), torch.Size([10000, 28, 28]))

In [17]:
tr_dl = torch.utils.data.DataLoader(
                          dataset=tr_dset,
                          batch_size=1024,
                          shuffle = True,
                          num_workers = 2)
tst_dl = torch.utils.data.DataLoader(
                          dataset=tst_dset,
                          batch_size=1024,
                          shuffle = False,
                          num_workers = 2)

In [18]:
xb, yb = next(iter(tst_dl))
xb.shape, yb.shape

(torch.Size([1024, 1, 28, 28]), torch.Size([1024]))

## Model and Opt

In [4]:
from miniai.conv import *

In [5]:
def get_model():
    return nn.Sequential(conv(1 ,8), conv(8 ,16), conv(16,32), conv(32,64),
                         conv(64,10, act=False), nn.Flatten())

In [21]:
get_model()

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (4): Conv2d(64, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (5): Flatten(start_dim=1, end_dim=-1)
)

In [22]:
get_model()(xb).shape

torch.Size([1024, 10])

In [23]:
model = get_model().to(device)

In [24]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

In [25]:
optimizer_mom = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9)

In [26]:
criterion = nn.CrossEntropyLoss()
criterion

CrossEntropyLoss()

## Training Loop

In [27]:
epochs = 5
n_total_steps = len(tr_dl)
n_total_steps

59

In [6]:
!pip install -q torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 10.6 MB/s eta 0:00:00


In [7]:
from torchmetrics.classification import MulticlassAccuracy

In [30]:
from torch import tensor

targ = tensor([2, 1, 0, 0])
prds = tensor([2, 1, 0, 1])
metric = MulticlassAccuracy(num_classes=3)
metric(prds, targ)

tensor(0.8333)

In [8]:
def train_step(model, data_loader,loss_fn, optimizer, accuracy_fn, device):
  train_loss, train_acc = 0,0
  for batch, (X,y) in enumerate(data_loader):
    X,y = X.to(device),y.to(device)
    y_pred = model(X)
    loss = loss_fn(y_pred, y)
    train_loss += loss
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  train_loss /= len(data_loader)
  train_acc /= len(data_loader)
  print(f"Train loss: {train_loss} | Train accuracy: {train_acc}%")

In [9]:
def test_step(data_loader, model, loss_fn, accuracy_fn, device):
  test_loss, test_acc = 0, 0
  model.eval()
  with torch.inference_mode():
    for X, y in data_loader:
      X, y = X.to(device), y.to(device)
      test_pred = model(X)
      test_loss += loss_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)

    # Adjust metrics and print out
    test_loss /= len(data_loader)
    test_acc /= len(data_loader)
    print(f"Test loss: {test_loss:.5f} | Test accuracy: {test_acc:.2f}%\n")

In [37]:
torch.manual_seed(42)

epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n---------")
  train_step(data_loader=tr_dl,
      model=model,
      loss_fn=criterion,
      optimizer=optimizer,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )
  test_step(data_loader=tst_dl,
      model=model,
      loss_fn=criterion,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )

Epoch: 0
---------
Train loss: 2.3003389835357666 | Train accuracy: 0.13782401382923126%
Test loss: 2.29623 | Test accuracy: 0.26%

Epoch: 1
---------
Train loss: 2.163888692855835 | Train accuracy: 0.3207462430000305%
Test loss: 1.29961 | Test accuracy: 0.60%

Epoch: 2
---------
Train loss: 0.9957460761070251 | Train accuracy: 0.6357304453849792%
Test loss: 0.85575 | Test accuracy: 0.66%

Epoch: 3
---------
Train loss: 0.7170441746711731 | Train accuracy: 0.723793089389801%
Test loss: 0.79683 | Test accuracy: 0.71%

Epoch: 4
---------
Train loss: 0.634416937828064 | Train accuracy: 0.7606388926506042%
Test loss: 0.70102 | Test accuracy: 0.74%



In [38]:
model.state_dict()

OrderedDict([('0.0.weight',
              tensor([[[[ 0.2946,  0.1661,  0.4074],
                        [ 0.3832,  0.2132,  0.3619],
                        [ 0.0044,  0.2698,  0.1861]]],
              
              
                      [[[ 0.4274,  0.7649, -0.0795],
                        [ 0.0686,  0.6966,  0.5101],
                        [ 0.3913,  0.5659,  0.0068]]],
              
              
                      [[[ 0.1639, -0.2808,  0.0931],
                        [-0.3447, -0.0912, -0.2201],
                        [-0.3234,  0.1768, -0.1962]]],
              
              
                      [[[ 0.1633,  0.0610,  0.0821],
                        [-0.0522,  0.1532, -0.2075],
                        [ 0.2617,  0.1287,  0.0173]]],
              
              
                      [[[ 0.1170,  0.3177, -0.0425],
                        [-0.0393, -0.1919, -0.1093],
                        [-0.2377,  0.1565, -0.3420]]],
              
              
                 

## Optimizer with momentum

In [39]:
model = get_model().to(device)
optimizer_mom = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [40]:
torch.manual_seed(42)

epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n---------")
  train_step(data_loader=tr_dl,
      model=model,
      loss_fn=criterion,
      optimizer=optimizer_mom,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )
  test_step(data_loader=tst_dl,
      model=model,
      loss_fn=criterion,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )

Epoch: 0
---------
Train loss: 2.3197696208953857 | Train accuracy: 0.1481204479932785%
Test loss: 2.29475 | Test accuracy: 0.10%

Epoch: 1
---------
Train loss: 2.0851387977600098 | Train accuracy: 0.25398877263069153%
Test loss: 1.23814 | Test accuracy: 0.55%

Epoch: 2
---------
Train loss: 1.7652733325958252 | Train accuracy: 0.36521580815315247%
Test loss: 1.23800 | Test accuracy: 0.50%

Epoch: 3
---------
Train loss: 0.7790383100509644 | Train accuracy: 0.7098100781440735%
Test loss: 0.67900 | Test accuracy: 0.74%

Epoch: 4
---------
Train loss: 0.5755404233932495 | Train accuracy: 0.7860608696937561%
Test loss: 0.54242 | Test accuracy: 0.80%



# 2. Data: huggingface

In [10]:
!pip install -q datasets

In [11]:
import torchvision.transforms.functional as TF,torch.nn.functional as F

In [12]:
from datasets import load_dataset

In [13]:
from miniai.datasets import *
from miniai.conv import *
from miniai.learner import *
from miniai.activations import *

In [15]:
x,y = 'image','label'
name = "fashion_mnist"
dsd = load_dataset(name)
bs = 1024

@inplace
def transformi(b): b[x] = [TF.to_tensor(o) for o in b[x]]

tds = dsd.with_transform(transformi)
dls = DataLoaders.from_dd(tds, bs, num_workers=2)
dt = dls.train

Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [17]:
type(dsd), type(tds), type(dls)

(datasets.dataset_dict.DatasetDict,
 datasets.dataset_dict.DatasetDict,
 miniai.datasets.DataLoaders)

In [12]:
xb,yb = next(iter(dt))
xb.shape, yb.shape

(torch.Size([1024, 1, 28, 28]), torch.Size([1024]))

## Model and opt

In [13]:
def get_model():
    return nn.Sequential(conv(1 ,8), conv(8 ,16), conv(16,32), conv(32,64),
                         conv(64,10, act=False), nn.Flatten())

get_model()

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (1): Sequential(
    (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
  )
  (4): Conv2d(64, 10, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (5): Flatten(start_dim=1, end_dim=-1)
)

In [14]:
get_model()(xb).shape

torch.Size([1024, 10])

In [56]:
model = get_model().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)
optimizer_mom = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
criterion = nn.CrossEntropyLoss()

## Training loop

In [16]:
!pip install -q torchmetrics
from torchmetrics.classification import MulticlassAccuracy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 8.7 MB/s eta 0:00:00


In [19]:
dsds = dls.valid
a,b = next(iter(dsds))
a.shape

torch.Size([2048, 1, 28, 28])

In [20]:
torch.manual_seed(42)

epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n---------")
  train_step(data_loader=dls.train,
      model=model,
      loss_fn=criterion,
      optimizer=optimizer,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )
  test_step(data_loader=dls.valid,
      model=model,
      loss_fn=criterion,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )

Epoch: 0
---------
Train loss: 2.2988195419311523 | Train accuracy: 0.13190743327140808%
Test loss: 2.28642 | Test accuracy: 0.13%

Epoch: 1
---------
Train loss: 1.756495714187622 | Train accuracy: 0.41380569338798523%
Test loss: 1.10228 | Test accuracy: 0.56%

Epoch: 2
---------
Train loss: 0.8644846081733704 | Train accuracy: 0.6808960437774658%
Test loss: 0.78751 | Test accuracy: 0.70%

Epoch: 3
---------
Train loss: 0.7048178315162659 | Train accuracy: 0.739458441734314%
Test loss: 0.66722 | Test accuracy: 0.75%

Epoch: 4
---------
Train loss: 0.6264652609825134 | Train accuracy: 0.7677029967308044%
Test loss: 0.59748 | Test accuracy: 0.78%



## optimizer with momentum

In [21]:
model = get_model().to(device)
optimizer_mom = torch.optim.SGD(model.parameters(), lr=0.2, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [22]:
torch.manual_seed(42)

epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n---------")
  train_step(data_loader=dls.train,
      model=model,
      loss_fn=criterion,
      optimizer=optimizer_mom,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )
  test_step(data_loader=dls.valid,
      model=model,
      loss_fn=criterion,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )

Epoch: 0
---------
Train loss: 2.3197684288024902 | Train accuracy: 0.1481204479932785%
Test loss: 2.29469 | Test accuracy: 0.10%

Epoch: 1
---------
Train loss: 2.2559239864349365 | Train accuracy: 0.15687434375286102%
Test loss: 2.30743 | Test accuracy: 0.10%

Epoch: 2
---------
Train loss: 2.303496837615967 | Train accuracy: 0.09999994188547134%
Test loss: 2.30287 | Test accuracy: 0.10%

Epoch: 3
---------
Train loss: 2.3029627799987793 | Train accuracy: 0.09999994188547134%
Test loss: 2.30286 | Test accuracy: 0.10%

Epoch: 4
---------
Train loss: 2.3030648231506348 | Train accuracy: 0.09999994188547134%
Test loss: 2.30296 | Test accuracy: 0.10%



# Experiments with huggingface data

In [23]:
dsd

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [24]:
@inplace
def transformi(b): b[xl] = [TF.to_tensor(o) for o in b[xl]]

bs = 1024
tds = dsd.with_transform(transformi)

In [25]:
tds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 60000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 10000
    })
})

In [27]:
DataLoaders.from_dd

<bound method DataLoaders.from_dd of <class 'miniai.datasets.DataLoaders'>>

In [30]:
tds["train"]

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [31]:
tds["train"].shape

(60000, 2)

In [ ]:
# torch.utils.data.DataLoader

In [43]:
x,y = 'image','label'
name = "fashion_mnist"
ds_train = load_dataset(name, split="train")
ds_test = load_dataset(name, split="test")

bs = 1024

In [20]:
import datasets

In [47]:
ds_test

Dataset({
    features: ['image', 'label'],
    num_rows: 10000
})

In [48]:
# ds_train.set_format(type="torch")
# ds_test.set_format(type="torch")
ds_train.set_format(type='torch', columns=['image', 'label'])
ds_test.set_format(type='torch', columns=['image', 'label'])

In [49]:
type(ds_train)

datasets.arrow_dataset.Dataset

In [50]:
dls_train = torch.utils.data.DataLoader(dataset=ds_train,
                                        shuffle=True,
                                        batch_size=bs,
                                        num_workers = 2)
dls_test = torch.utils.data.DataLoader(dataset=ds_test,
                                       shuffle=False,
                                       batch_size=bs,
                                       num_workers = 2)

In [51]:
type(dls_train)

torch.utils.data.dataloader.DataLoader

In [52]:
dls_train.dataset.shape

(60000, 2)

In [53]:
dls_train.dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 60000
})

In [59]:
dls_train.dataset["image"].shape

torch.Size([60000, 28, 28])

In [60]:
dls_train["image"]

TypeError: ignored

In [54]:
xb,yb = next(iter(dls_train))[]
xb.shape

AttributeError: ignored

In [57]:
torch.manual_seed(42)

epochs = 5
for epoch in range(epochs):
  print(f"Epoch: {epoch}\n---------")
  train_step(data_loader=dls_train,
      model=model,
      loss_fn=criterion,
      optimizer=optimizer,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )
  test_step(data_loader=dls_valid,
      model=model,
      loss_fn=criterion,
      accuracy_fn=MulticlassAccuracy(num_classes=10).to(device),
      device = device
  )

Epoch: 0
---------


AttributeError: ignored